Back to **[Fan](https://fanwangecon.github.io/)**'s R4Econ Homepage **[Table of Content](https://fanwangecon.github.io/R4Econ/)**

# Constrained Share Parameters to Unconstrained Parameters

Sometimes, the parameters we are optimizing over are constrained, we might be optimizing by choosing $a,b,c$. They sum up ot $1$, and each is greater or equal to $0$. 

$$ a + b + c = 1$$

We can solve the problem as a constrained optimization problem.

We can also restate $a$, $b$ and $c$ as unconstrained parameters, and use the functions below to make sure the unconstrained parameters could be transformed back.

In [44]:
library(tidyverse)

## Constrained Fraction/Shares to Unconstrained Parameters

The original fractional variables are:
$$ \sum_i x_i = 1$$
$$ x_i \ge 0 $$
We transform to $\hat{x_i}$, using this transformation function
$$ G_i - \frac{G_i}{1+exp(\hat{x_i})} = x_i$$
Where $G_1 = 1$, and $$G_i = 1 - \sum_{i=1}^{i-1} x_i $$
Where $x$ is the original fractional variable that is constrained.

The optimizer is optimizing over $\hat{x_i}$, which are unconstrained, but the model uses the transformed constrained $x_i$. During optimization, optimizer changing $\hat{x_i}$. Given that $x_i$ sum up to a fixed aggregate, we optimize over 1 less $i$ than the total number of fractions. 

Note that the total sum of $x_i$ do not have to be $1$. 

## Transformer Function

The function below executes the transformation from unconstrained maximizers back to constrained.

In [45]:
# Convert one by one
f_frac_asymp <- function(x, sca.total.frac=1) {
    sca.subsidy.frac <- sca.total.frac - sca.total.frac/(1+exp(x))
    return(sca.subsidy.frac)
}

In [46]:
# Vector of Fractions from Asymptotic Transform
f_frac_asymp_vec <- function(x) {
    # inputs: x <- c(1, 1, 1), these are the x_hats, the unconstrained transformed
    # outputs: the return output of the function are the original fractionos
    
    sca.x.counter <- 1
    sca.total.frac <- 1
    vec.subsidy.frac <- numeric(length(x)+1)
    while(sca.x.counter <= length(x)) {
        sca.x.cur <- x[sca.x.counter]
        sca.subsidy.frac <- f_frac_asymp(sca.x.cur, sca.total.frac)
        vec.subsidy.frac[sca.x.counter] <- sca.subsidy.frac
        sca.total.frac <- sca.total.frac - sca.subsidy.frac
        sca.x.counter <- sca.x.counter + 1
    }
    
    vec.subsidy.frac[length(x)+1] <- sca.total.frac
    
    return(vec.subsidy.frac)
}

In [47]:
# Testing
f_frac_asymp_vec(c(-1, -0.5, 0))
f_frac_asymp_vec(c(1, 1, 1))
f_frac_asymp_vec(c(1, 2, 1))
f_frac_asymp_vec(c(-25, -5, -1))

[1] 0.2689414 0.2760043 0.2275271 0.2275271

[1] 0.73105858 0.19661193 0.05287709 0.01945240

[1] 0.731058579 0.236882818 0.023436717 0.008621886

[1] 1.388800e-11 6.692851e-03 2.671414e-01 7.261657e-01

In [48]:
# Testing
f_frac_asymp_vec(c(1, 1, 1, 1))
f_frac_asymp_vec(c(2, 2, 2, 2))

[1] 0.731058579 0.196611933 0.052877093 0.014220840 0.005231555

[1] 0.8807970780 0.1049935854 0.0125155422 0.0014918892 0.0002019052

## Initializer Function

We will need to initialize parameters. They could be drawn randomly, they could be uniform fractions, or normal (more weight in the middle), or exponential (higher fraction at one end, monotonic)

In [49]:
# Fractional Estimands Initializer Function
f_subsidy_frac <- function(n, type, seed=123) {
    
    if (type == 'rand') {
        set.seed(seed)
        vec.draws <- runif(n)
        vec.prob <- vec.draws/sum(vec.draws)
    }
    
    if (type == 'unif') {
        vec.prob <- numeric(n) + 1/n 
    }
    
    if (type == 'dexp') {
        bin <- 1/(n)
        vec.quantiles <- seq(bin/2, 1, bin)
        vec.points <- qexp(vec.quantiles)
        vec.prob <- vec.points/sum(vec.points)
        vec.prob <- sort(vec.prob)
    }    

    if (type == 'dnorm') {
        bin <- 1/(n)
        vec.quantiles <- seq(bin/2, 1, bin)
        vec.points <- qnorm(vec.quantiles)
        vec.prob <- dnorm(vec.points)/sum(dnorm(vec.points))
    }    

    return((vec.prob))
}
# Different Probability Strucgture
cbind(tibble(type='rand', frac=f_subsidy_frac(5, 'rand')),
      tibble(type='unif', frac=f_subsidy_frac(5, 'unif')),
      tibble(type='dexp', frac=f_subsidy_frac(5, 'dexp')),
      tibble(type='dnorm', frac=f_subsidy_frac(5, 'dnorm')))

type,frac,type,frac,type,frac,type,frac
rand,0.08692491,unif,0.2,dexp,0.02260111,dnorm,0.1214249
rand,0.23827784,unif,0.2,dexp,0.07651111,dnorm,0.2405638
rand,0.12361982,unif,0.2,dexp,0.14868849,dnorm,0.2760227
rand,0.26690614,unif,0.2,dexp,0.25826680,dnorm,0.2405638
rand,0.28427129,unif,0.2,dexp,0.49393249,dnorm,0.1214249
